In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
def read_csvs(path):
    data = pd.DataFrame()
    for root, dir, files in os.walk(path):
        for file in files:
            if file.endswith(".csv"):
                data = pd.concat([data, pd.read_csv(os.path.join(root, file))])
    return data

df = read_csvs("out/csv/squat")

In [3]:
x = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [4]:
x.shape

(3585, 12)

In [5]:
# x = x.reshape(len(x), 1, x.shape[1])

In [6]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x, y, train_size=0.8, test_size=0.2, random_state=1)

In [8]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, Reshape
from keras.optimizers import Adam

model = Sequential([
    Reshape((1, x.shape[1]), input_shape=(x.shape[1],)), # 12 is the number of features
    LSTM(128, activation='relu', return_sequences=True),
    LSTM(64, activation='relu'),
    Dense(32, activation='sigmoid'),
    Dense(1, activation='sigmoid')
])

opt_adam = Adam(learning_rate=1e-3)

model.compile(optimizer=opt_adam, loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
model.fit(x_train, y_train, epochs=200, validation_data=(x_test, y_test))

Epoch 1/200
90/90 [==============================] - 6s 19ms/step - loss: 0.6979 - accuracy: 0.4969 - val_loss: 0.6935 - val_accuracy: 0.4923
Epoch 2/200
90/90 [==============================] - 1s 7ms/step - loss: 0.6931 - accuracy: 0.5084 - val_loss: 0.6922 - val_accuracy: 0.6374
Epoch 3/200
90/90 [==============================] - 1s 7ms/step - loss: 0.6902 - accuracy: 0.5718 - val_loss: 0.6847 - val_accuracy: 0.5495
Epoch 4/200
90/90 [==============================] - 1s 7ms/step - loss: 0.6593 - accuracy: 0.6015 - val_loss: 0.6611 - val_accuracy: 0.6388
Epoch 5/200
90/90 [==============================] - 1s 7ms/step - loss: 0.6292 - accuracy: 0.6430 - val_loss: 0.6191 - val_accuracy: 0.6457
Epoch 6/200
90/90 [==============================] - 1s 7ms/step - loss: 0.6203 - accuracy: 0.6527 - val_loss: 0.6182 - val_accuracy: 0.6583
Epoch 7/200
90/90 [==============================] - 1s 7ms/step - loss: 0.6296 - accuracy: 0.6475 - val_loss: 0.6362 - val_accuracy: 0.6179
Epoch 8/200


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 1, 12)             0         
                                                                 
 lstm (LSTM)                 (None, 1, 128)            72192     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 123,713
Trainable params: 123,713
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.save('out/models/model3.h5')

In [47]:
from keras.models import save_model

save_model(model, 'out/models/android.h5')

In [48]:
from tensorflow import lite
converter = lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
  lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tfmodel = converter.convert()
open('out/models/android.tflite', 'wb').write(tfmodel)

INFO:tensorflow:Assets written to: C:\Users\SIDDHA~1\AppData\Local\Temp\tmpnysh_pl0\assets


INFO:tensorflow:Assets written to: C:\Users\SIDDHA~1\AppData\Local\Temp\tmpnysh_pl0\assets


390348